# Loading the data sets

Initially we load all our three base files:

1. Mortality
2. Population
3. Shipment

In [1]:
import pandas as pd 
import numpy as np

# Read in data
mortality = pd.read_csv('../10_Clean_Data/mortality/mortality.csv')

population = pd.read_csv('../10_Clean_Data/population/final_population_data.csv')

shipments = pd.read_parquet('../01_Source_Data/shipments/shipments_data.gzip')

# First we reshape the population because population had a column for each year
population_melted = population.melt(id_vars=["GISJOIN", "STATE_CODE", "COUNTY_CODE", "COUNTY", "STATE"],
          var_name="Year",
          value_name="Population")
population_melted.head()


,GISJOIN,STATE_CODE,COUNTY_CODE,COUNTY,STATE,Year,Population
0,G0100010,1,1,Autauga County,Alabama,2003,46348
1,G0100030,1,3,Baldwin County,Alabama,2003,151574
2,G0100050,1,5,Barbour County,Alabama,2003,28805
3,G0100070,1,7,Bibb County,Alabama,2003,21224
4,G0100090,1,9,Blount County,Alabama,2003,54195


# Some Data Cleaning

In [2]:
#in population, change District Of Columbia to District of Columbia for STATE (capital O to lowercase o) -> spent hours trying to figure out why it wasn't merging
population_melted['STATE'] = population_melted['STATE'].replace('District Of Columbia', 'District of Columbia')

In [3]:
#remove Alaska from mortality,shipment and population data
mortality['State'] = mortality['State'].str.strip()
mortality = mortality[mortality['State'] != 'AK']
shipments['BUYER_STATE'] = shipments['BUYER_STATE'].str.strip()
shipments = shipments[shipments['BUYER_STATE'] != 'AK']
population_melted['STATE'] = population_melted['STATE'].str.strip()
population_melted = population_melted[population_melted['STATE'] != 'Alaska']

In [4]:
#also remove PR (Puerto Rico) from shipment data
shipments = shipments[shipments['BUYER_STATE'] != 'PR']

In [5]:
#value counts for county and state pair for mortality - just to see if there are any repitions. 13 count means there are 13 years of data for that county. Some counties dont have data in all years
(mortality.groupby(['County', 'State']).size()).sort_values(ascending=False)

County                   State
El Paso County           CO       13
East Baton Rouge Parish  LA       13
Durham County            NC       13
Salt Lake County         UT       13
DuPage County            IL       13
                                  ..
Lawrence County          AL        1
Tipton County            TN        1
McKinley County          NM        1
Delaware County          OK        1
Van Zandt County         TX        1
Length: 1039, dtype: int64

In [6]:
mortality.head()

,County,Year,Deaths,State
0,Acadia Parish,2003,11,LA
1,Ada County,2003,17,ID
2,Adams County,2003,42,CO
3,Aiken County,2003,10,SC
4,Alachua County,2003,11,FL


In [7]:
shipments.head()

,BUYER_COUNTY,BUYER_STATE,Year,MME,CALC_BASE_WT_IN_GM
0,ABBEVILLE,SC,2006,3.136215e+06,2506.084390
1,ABBEVILLE,SC,2007,3.232603e+06,2623.718375
2,ABBEVILLE,SC,2008,3.070698e+06,2574.677256
3,ABBEVILLE,SC,2009,3.827607e+06,3110.779538
4,ABBEVILLE,SC,2010,4.612935e+06,3695.582848


In [8]:
#check for nas 
population_melted.isna().sum()
shipments.isna().sum()
mortality.isna().sum()


County    0
Year      0
Deaths    0
State     0
dtype: int64

In [9]:
# remove the second word from COUNTY variable and make it lowercase
#population_melted['COUNTY'] = population_melted['COUNTY'].str.replace(' county', '', case=False).str.lower()
population_melted['COUNTY'] = population_melted['COUNTY'].str.replace(' county', '', case=False).str.replace(' parish', '', case=False).str.lower()
population_melted.head()

,GISJOIN,STATE_CODE,COUNTY_CODE,COUNTY,STATE,Year,Population
0,G0100010,1,1,autauga,Alabama,2003,46348
1,G0100030,1,3,baldwin,Alabama,2003,151574
2,G0100050,1,5,barbour,Alabama,2003,28805
3,G0100070,1,7,bibb,Alabama,2003,21224
4,G0100090,1,9,blount,Alabama,2003,54195


In [10]:
# remove the second word from BUYER_COUNTY variable in shipments and make it lowercase
shipments['BUYER_COUNTY'] = shipments['BUYER_COUNTY'].str.replace(' county', '', case=False).str.replace(' parish', '', case=False).str.lower()
shipments.head()

,BUYER_COUNTY,BUYER_STATE,Year,MME,CALC_BASE_WT_IN_GM
0,abbeville,SC,2006,3.136215e+06,2506.084390
1,abbeville,SC,2007,3.232603e+06,2623.718375
2,abbeville,SC,2008,3.070698e+06,2574.677256
3,abbeville,SC,2009,3.827607e+06,3110.779538
4,abbeville,SC,2010,4.612935e+06,3695.582848


In [11]:
#remove words like county and parish from mortality county variable and make it lowercase because its not in other datasets
mortality['County'] = mortality['County'].str.replace(' county', '', case=False).str.replace(' parish', '', case=False).str.lower()
mortality.head()

,County,Year,Deaths,State
0,acadia,2003,11,LA
1,ada,2003,17,ID
2,adams,2003,42,CO
3,aiken,2003,10,SC
4,alachua,2003,11,FL


In [12]:
#Since we are merging on county and state, we need to make sure that the county and state names are the same across all datasets.
#Mapping state names to abbreviations

state_dict = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
state_dict = dict(map(reversed, state_dict.items()))


In [13]:
#add a column to mortality to match the state abbreviations
#remove whitespace from state column before the state name
mortality['State Name'] = mortality['State'].map(state_dict)

In [14]:
mortality.head()

,County,Year,Deaths,State,State Name
0,acadia,2003,11,LA,Louisiana
1,ada,2003,17,ID,Idaho
2,adams,2003,42,CO,Colorado
3,aiken,2003,10,SC,South Carolina
4,alachua,2003,11,FL,Florida


In [15]:
#do the same for shipments
shipments['State Name'] = shipments['BUYER_STATE'].map(state_dict)

In [16]:
shipments.head()

,BUYER_COUNTY,BUYER_STATE,Year,MME,CALC_BASE_WT_IN_GM,State Name
0,abbeville,SC,2006,3.136215e+06,2506.084390,South Carolina
1,abbeville,SC,2007,3.232603e+06,2623.718375,South Carolina
2,abbeville,SC,2008,3.070698e+06,2574.677256,South Carolina
3,abbeville,SC,2009,3.827607e+06,3110.779538,South Carolina
4,abbeville,SC,2010,4.612935e+06,3695.582848,South Carolina


## NOW WE HAVE ALL 3 DATASETS AND NECESSARY DATA CLEANING AND ADDITIONS ARE DONE. LETS CHECK ALL 3 now

In [17]:
#shape of all three
print(mortality.shape)
print(shipments.shape)
print(population_melted.shape)

(7888, 5)
(41845, 6)
(40482, 7)


In [18]:
#number of states in each
print(len(mortality['State Name'].unique()))
print(len(shipments['State Name'].unique()))
print(len(population_melted['STATE'].unique()))

50
54
50


In [19]:
#find the difference in states between mortality and shipments
set(shipments['State Name'].unique()) - set(mortality['State Name'].unique())
#find the difference in states between mortality and population
#set(population_melted['STATE'].unique()) - set(mortality['State Name'].unique())

{'Guam', 'Northern Mariana Islands', 'U.S. Virgin Islands', nan}

In [20]:
#remove Guam, Northern Mariana Islands, U.S. Virgin Islands from shipments
shipments = shipments[shipments['State Name'] != 'Guam']
shipments = shipments[shipments['State Name'] != 'Northern Mariana Islands']
shipments = shipments[shipments['State Name'] != 'U.S. Virgin Islands']


In [21]:
shipments.head()

,BUYER_COUNTY,BUYER_STATE,Year,MME,CALC_BASE_WT_IN_GM,State Name
0,abbeville,SC,2006,3.136215e+06,2506.084390,South Carolina
1,abbeville,SC,2007,3.232603e+06,2623.718375,South Carolina
2,abbeville,SC,2008,3.070698e+06,2574.677256,South Carolina
3,abbeville,SC,2009,3.827607e+06,3110.779538,South Carolina
4,abbeville,SC,2010,4.612935e+06,3695.582848,South Carolina


In [22]:
mortality.head(5)

,County,Year,Deaths,State,State Name
0,acadia,2003,11,LA,Louisiana
1,ada,2003,17,ID,Idaho
2,adams,2003,42,CO,Colorado
3,aiken,2003,10,SC,South Carolina
4,alachua,2003,11,FL,Florida


In [23]:
population_melted.head()

,GISJOIN,STATE_CODE,COUNTY_CODE,COUNTY,STATE,Year,Population
0,G0100010,1,1,autauga,Alabama,2003,46348
1,G0100030,1,3,baldwin,Alabama,2003,151574
2,G0100050,1,5,barbour,Alabama,2003,28805
3,G0100070,1,7,bibb,Alabama,2003,21224
4,G0100090,1,9,blount,Alabama,2003,54195


In [24]:
#checking number of unique counties state pairs in each
print(len(mortality.groupby(['County', 'State Name'])))
print(len(shipments.groupby(['BUYER_COUNTY', 'State Name'])))
print(len(population_melted.groupby(['COUNTY', 'STATE'])))

1039
3032
3114


In [25]:
#years in each, make year in shipments int
mortality['Year'] = mortality['Year'].astype(int)
shipments['Year'] = shipments['Year'].astype(int)
population_melted['Year'] = population_melted['Year'].astype(int)


print("mortality:",  mortality['Year'].unique())
print("shipments:", shipments['Year'].unique())
print("population:", population_melted['Year'].unique())




mortality: [2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015]
shipments: [2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019]
population: [2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015]


# Merging Process

In [26]:
#in mortality, the county in Indiana called 'laporte' is named as 'la porte', se we need to change that
mortality['County'] = mortality['County'].replace('la porte', 'laporte')

#also in mortality, the county in Pennslyvania called 'mckean' is named as 'mc kean', se we need to change that
mortality['County'] = mortality['County'].replace('mc kean', 'mckean')

In [27]:
#now merge the three datasets
#first merge mortality and population outer
mortality_population = pd.merge(mortality,population_melted, how='outer', left_on=['County', 'State Name', 'Year'], right_on=['COUNTY', 'STATE', 'Year'],validate="1:1",
indicator=True)

mortality_population._merge.value_counts()

_merge
right_only    32594
both           7888
left_only         0
Name: count, dtype: int64

In [28]:
#total unique counties in florida
len(mortality_population[mortality_population['STATE'] == 'Florida']['COUNTY'].unique())

67

In [29]:
#check the left only
mortality_population[mortality_population['_merge'] == 'left_only'] #none will be returned for this
#There is no left_only merge, which means that the only missing data we have now is in the mortality dataset because of the missing counties in the mortality dataset (less than 10). We will impute them later on.


,County,Year,Deaths,State,State Name,GISJOIN,STATE_CODE,COUNTY_CODE,COUNTY,STATE,Population,_merge


In [30]:
mortality_population

,County,Year,Deaths,State,State Name,GISJOIN,STATE_CODE,COUNTY_CODE,COUNTY,STATE,Population,_merge
0,acadia,2003,11.0,LA,Louisiana,G2200010,22,1,acadia,Louisiana,59209,both
1,ada,2003,17.0,ID,Idaho,G1600010,16,1,ada,Idaho,325482,both
2,adams,2003,42.0,CO,Colorado,G0800010,8,1,adams,Colorado,381370,both
3,aiken,2003,10.0,SC,South Carolina,G4500030,45,3,aiken,South Carolina,146903,both
4,alachua,2003,11.0,FL,Florida,G1200010,12,1,alachua,Florida,221717,both
...,...,...,...,...,...,...,...,...,...,...,...,...
40477,NaN,2015,NaN,NaN,NaN,G5600370,56,37,sweetwater,Wyoming,45604,right_only
40478,NaN,2015,NaN,NaN,NaN,G5600390,56,39,teton,Wyoming,21800,right_only
40479,NaN,2015,NaN,NaN,NaN,G5600410,56,41,uinta,Wyoming,21154,right_only
40480,NaN,2015,NaN,NaN,NaN,G5600430,56,43,washakie,Wyoming,8586,right_only


In [31]:
#mortality_population[mortality_population['County'] == 'district of columbia']

In [32]:
#check counties in alabama in mortality_population
mortality_population[mortality_population['State Name'] == 'Alabama']['County'].unique()

array(['baldwin', 'jefferson', 'mobile', 'madison', 'montgomery',
       'morgan', 'tuscaloosa', 'walker', 'chilton', 'shelby', 'calhoun',
       'houston', 'lauderdale', 'st. clair', 'blount', 'cullman',
       'etowah', 'limestone', 'escambia', 'lawrence', 'lee', 'dekalb',
       'jackson', 'marshall', 'talladega', 'franklin'], dtype=object)

In [33]:
#change st and st. to saint in mortality_population
mortality_population['COUNTY'] = mortality_population['COUNTY'].str.replace('st ', 'saint ', case=False)
mortality_population['COUNTY'] = mortality_population['COUNTY'].str.replace('st. ', 'saint ', case=False)

#change desoto to de soto in mortality_population
mortality_population['COUNTY'] = mortality_population['COUNTY'].str.replace('desoto', 'de soto', case=False)
mortality_population['COUNTY'] = mortality_population['COUNTY'].str.replace('dewitt', 'de witt', case=False)



In [34]:
#check counties in florida in mortality_population
mortality_population[mortality_population['STATE'] == 'Florida']['COUNTY'].unique()

array(['alachua', 'bay', 'brevard', 'broward', 'citrus', 'clay',
       'collier', 'duval', 'escambia', 'hernando', 'highlands',
       'hillsborough', 'indian river', 'lake', 'lee', 'leon', 'manatee',
       'marion', 'martin', 'miami-dade', 'monroe', 'okaloosa', 'orange',
       'osceola', 'palm beach', 'pasco', 'pinellas', 'polk', 'santa rosa',
       'sarasota', 'seminole', 'saint lucie', 'volusia', 'charlotte',
       'nassau', 'putnam', 'saint johns', 'columbia', 'walton',
       'okeechobee', 'flagler', 'levy', 'sumter', 'baker', 'bradford',
       'calhoun', 'de soto', 'dixie', 'franklin', 'gadsden', 'gilchrist',
       'glades', 'gulf', 'hamilton', 'hardee', 'hendry', 'holmes',
       'jackson', 'jefferson', 'lafayette', 'liberty', 'madison',
       'suwannee', 'taylor', 'union', 'wakulla', 'washington'],
      dtype=object)

In [35]:
#find all counties in Louisiana in mortality_population
len(mortality_population[mortality_population['State Name'] == 'Florida']['COUNTY'].unique())

43

In [36]:
#total unique county-state in mortality_population
len(mortality_population.groupby(['COUNTY', 'STATE']))


3114

In [37]:
#change de kalb dekalb in shipments
shipments['BUYER_COUNTY'] = shipments['BUYER_COUNTY'].replace('de kalb', 'dekalb')

#change st and st. to saint in shipments
shipments['BUYER_COUNTY'] = shipments['BUYER_COUNTY'].replace('st. ', 'saint ')
shipments['BUYER_COUNTY'] = shipments['BUYER_COUNTY'].replace('st ', 'saint ')

shipments['BUYER_COUNTY'] = shipments['BUYER_COUNTY'].str.replace('desoto', 'de soto', case=False)
shipments['BUYER_COUNTY'] = shipments['BUYER_COUNTY'].str.replace('dewitt', 'de witt', case=False)
shipments['BUYER_COUNTY'] = shipments['BUYER_COUNTY'].str.replace('la porte', 'laporte', case=False)




In [38]:
#show where shipments is null for State Name
shipments[shipments['State Name'].isnull()]

#remove this state (PW) from shipments
shipments = shipments[shipments['State Name'].notna()]

In [39]:
shipments.isna().sum()

BUYER_COUNTY          0
BUYER_STATE           0
Year                  0
MME                   0
CALC_BASE_WT_IN_GM    0
State Name            0
dtype: int64

In [40]:
shipments.groupby(['BUYER_COUNTY', 'State Name', 'Year']).size().reset_index(name='count').sort_values(by='count', ascending=False)

,BUYER_COUNTY,State Name,Year,count
0,abbeville,South Carolina,2006,1
27859,northampton,Virginia,2008,1
27841,northampton,North Carolina,2018,1
27842,northampton,North Carolina,2019,1
27843,northampton,Pennsylvania,2006,1
...,...,...,...,...
13925,gilmer,West Virginia,2008,1
13926,gilmer,West Virginia,2009,1
13927,gilmer,West Virginia,2010,1
13928,gilmer,West Virginia,2011,1


In [41]:
#merge shipments and mortality_population
mortality_population_shipments = pd.merge(mortality_population,shipments, how='outer', left_on=['COUNTY', 'STATE', 'Year'], right_on=['BUYER_COUNTY', 'State Name', 'Year'], indicator = 'merge_indicator')

mortality_population_shipments['merge_indicator'].value_counts()

merge_indicator
both          29672
right_only    12098
left_only     10810
Name: count, dtype: int64

In [42]:
mortality_population_shipments[mortality_population_shipments.merge_indicator == "left_only"]['Year'].value_counts()

Year
2003    3114
2004    3114
2005    3114
2015     154
2008     152
2009     152
2010     152
2007     147
2011     144
2013     143
2006     142
2014     142
2012     140
Name: count, dtype: int64

In [43]:
#find de kalb in mortality_population

In [44]:
#give me all counties in Indiana in shipments
shipments[shipments['State Name'] == 'Indiana']['BUYER_COUNTY'].unique()

array(['adams', 'allen', 'bartholomew', 'benton', 'blackford', 'boone',
       'brown', 'carroll', 'cass', 'clark', 'clay', 'clinton', 'crawford',
       'daviess', 'dekalb', 'dearborn', 'decatur', 'delaware', 'dubois',
       'elkhart', 'fayette', 'floyd', 'fountain', 'franklin', 'fulton',
       'gibson', 'grant', 'greene', 'hamilton', 'hancock', 'harrison',
       'hendricks', 'henry', 'howard', 'huntington', 'jackson', 'jasper',
       'jay', 'jefferson', 'jennings', 'johnson', 'knox', 'kosciusko',
       'laporte', 'lagrange', 'lake', 'lawrence', 'madison', 'marion',
       'marshall', 'martin', 'miami', 'monroe', 'montgomery', 'morgan',
       'newton', 'noble', 'ohio', 'orange', 'owen', 'parke', 'perry',
       'pike', 'porter', 'posey', 'pulaski', 'putnam', 'randolph',
       'ripley', 'rush', 'scott', 'shelby', 'spencer', 'st joseph',
       'starke', 'steuben', 'sullivan', 'switzerland', 'tippecanoe',
       'tipton', 'union', 'vanderburgh', 'vermillion', 'vigo', 'wabash',
  

In [45]:
#check the right onlys before 2015
mortality_population_shipments[(mortality_population_shipments.merge_indicator == "right_only") & (mortality_population_shipments['Year'] < 2015)]

#get the unique county names from above
mortality_population_shipments[(mortality_population_shipments.merge_indicator == "right_only") & (mortality_population_shipments['Year'] < 2015)]['BUYER_COUNTY'].unique()

array(['east baton rouge', 'east carroll', 'east feliciana', 'la salle',
       'obrien', 'prince georges', 'queen annes', 'radford',
       'saint marys', 'sainte genevieve', 'salem', 'st john the baptist',
       'st joseph', 'west baton rouge', 'west carroll', 'west feliciana'],
      dtype=object)

In [46]:
#we can just remove the above and also remove data after 2015
mortality_population_shipments = mortality_population_shipments[(mortality_population_shipments['Year'] <= 2015) & (mortality_population_shipments.merge_indicator != "right_only")]

In [47]:
mortality_population_shipments

,County,Year,Deaths,State,State Name_x,GISJOIN,STATE_CODE,COUNTY_CODE,COUNTY,STATE,Population,_merge,BUYER_COUNTY,BUYER_STATE,MME,CALC_BASE_WT_IN_GM,State Name_y,merge_indicator
0,acadia,2003,11.0,LA,Louisiana,G2200010,22.0,1.0,acadia,Louisiana,59209.0,both,NaN,NaN,NaN,NaN,NaN,left_only
1,ada,2003,17.0,ID,Idaho,G1600010,16.0,1.0,ada,Idaho,325482.0,both,NaN,NaN,NaN,NaN,NaN,left_only
2,adams,2003,42.0,CO,Colorado,G0800010,8.0,1.0,adams,Colorado,381370.0,both,NaN,NaN,NaN,NaN,NaN,left_only
3,aiken,2003,10.0,SC,South Carolina,G4500030,45.0,3.0,aiken,South Carolina,146903.0,both,NaN,NaN,NaN,NaN,NaN,left_only
4,alachua,2003,11.0,FL,Florida,G1200010,12.0,1.0,alachua,Florida,221717.0,both,NaN,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40477,NaN,2015,NaN,NaN,NaN,G5600370,56.0,37.0,sweetwater,Wyoming,45604.0,right_only,sweetwater,WY,2.412509e+07,17488.438403,Wyoming,both
40478,NaN,2015,NaN,NaN,NaN,G5600390,56.0,39.0,teton,Wyoming,21800.0,right_only,teton,WY,3.343610e+06,2609.264725,Wyoming,both
40479,NaN,2015,NaN,NaN,NaN,G5600410,56.0,41.0,uinta,Wyoming,21154.0,right_only,uinta,WY,1.135771e+07,8459.162985,Wyoming,both
40480,NaN,2015,NaN,NaN,NaN,G5600430,56.0,43.0,washakie,Wyoming,8586.0,right_only,washakie,WY,7.700956e+06,5477.434225,Wyoming,both


In [48]:
#clean up and only keep relevant columns
mortality_population_shipments = mortality_population_shipments[['COUNTY', "STATE", 'State Name_x', 'Year', 'Deaths', 'Population', 'BUYER_COUNTY', 'BUYER_STATE', 'MME', 'CALC_BASE_WT_IN_GM', 'merge_indicator']]

In [49]:
mortality_population_shipments

,COUNTY,STATE,State Name_x,Year,Deaths,Population,BUYER_COUNTY,BUYER_STATE,MME,CALC_BASE_WT_IN_GM,merge_indicator
0,acadia,Louisiana,Louisiana,2003,11.0,59209.0,NaN,NaN,NaN,NaN,left_only
1,ada,Idaho,Idaho,2003,17.0,325482.0,NaN,NaN,NaN,NaN,left_only
2,adams,Colorado,Colorado,2003,42.0,381370.0,NaN,NaN,NaN,NaN,left_only
3,aiken,South Carolina,South Carolina,2003,10.0,146903.0,NaN,NaN,NaN,NaN,left_only
4,alachua,Florida,Florida,2003,11.0,221717.0,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
40477,sweetwater,Wyoming,NaN,2015,NaN,45604.0,sweetwater,WY,2.412509e+07,17488.438403,both
40478,teton,Wyoming,NaN,2015,NaN,21800.0,teton,WY,3.343610e+06,2609.264725,both
40479,uinta,Wyoming,NaN,2015,NaN,21154.0,uinta,WY,1.135771e+07,8459.162985,both
40480,washakie,Wyoming,NaN,2015,NaN,8586.0,washakie,WY,7.700956e+06,5477.434225,both


In [50]:
#remove la prte and mc kean county since we dont have population data for them so better to clean
mortality_population_shipments = mortality_population_shipments[mortality_population_shipments['COUNTY'] != 'la porte']
mortality_population_shipments = mortality_population_shipments[mortality_population_shipments['COUNTY'] != 'mc kean']

mortality_population_shipments

,COUNTY,STATE,State Name_x,Year,Deaths,Population,BUYER_COUNTY,BUYER_STATE,MME,CALC_BASE_WT_IN_GM,merge_indicator
0,acadia,Louisiana,Louisiana,2003,11.0,59209.0,NaN,NaN,NaN,NaN,left_only
1,ada,Idaho,Idaho,2003,17.0,325482.0,NaN,NaN,NaN,NaN,left_only
2,adams,Colorado,Colorado,2003,42.0,381370.0,NaN,NaN,NaN,NaN,left_only
3,aiken,South Carolina,South Carolina,2003,10.0,146903.0,NaN,NaN,NaN,NaN,left_only
4,alachua,Florida,Florida,2003,11.0,221717.0,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
40477,sweetwater,Wyoming,NaN,2015,NaN,45604.0,sweetwater,WY,2.412509e+07,17488.438403,both
40478,teton,Wyoming,NaN,2015,NaN,21800.0,teton,WY,3.343610e+06,2609.264725,both
40479,uinta,Wyoming,NaN,2015,NaN,21154.0,uinta,WY,1.135771e+07,8459.162985,both
40480,washakie,Wyoming,NaN,2015,NaN,8586.0,washakie,WY,7.700956e+06,5477.434225,both


In [51]:
merged = mortality_population_shipments

In [52]:
merged
#we still havent removed 2003,2004,2005 data (as that data is not in shipments) but we might need it later


,COUNTY,STATE,State Name_x,Year,Deaths,Population,BUYER_COUNTY,BUYER_STATE,MME,CALC_BASE_WT_IN_GM,merge_indicator
0,acadia,Louisiana,Louisiana,2003,11.0,59209.0,NaN,NaN,NaN,NaN,left_only
1,ada,Idaho,Idaho,2003,17.0,325482.0,NaN,NaN,NaN,NaN,left_only
2,adams,Colorado,Colorado,2003,42.0,381370.0,NaN,NaN,NaN,NaN,left_only
3,aiken,South Carolina,South Carolina,2003,10.0,146903.0,NaN,NaN,NaN,NaN,left_only
4,alachua,Florida,Florida,2003,11.0,221717.0,NaN,NaN,NaN,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...
40477,sweetwater,Wyoming,NaN,2015,NaN,45604.0,sweetwater,WY,2.412509e+07,17488.438403,both
40478,teton,Wyoming,NaN,2015,NaN,21800.0,teton,WY,3.343610e+06,2609.264725,both
40479,uinta,Wyoming,NaN,2015,NaN,21154.0,uinta,WY,1.135771e+07,8459.162985,both
40480,washakie,Wyoming,NaN,2015,NaN,8586.0,washakie,WY,7.700956e+06,5477.434225,both


# Imputation for death rate (normalized)

In [53]:
#calculate death rate

merged['Death Rate'] = merged['Deaths']/merged['Population']

In [54]:
merged

,COUNTY,STATE,State Name_x,Year,Deaths,Population,BUYER_COUNTY,BUYER_STATE,MME,CALC_BASE_WT_IN_GM,merge_indicator,Death Rate
0,acadia,Louisiana,Louisiana,2003,11.0,59209.0,NaN,NaN,NaN,NaN,left_only,0.000186
1,ada,Idaho,Idaho,2003,17.0,325482.0,NaN,NaN,NaN,NaN,left_only,0.000052
2,adams,Colorado,Colorado,2003,42.0,381370.0,NaN,NaN,NaN,NaN,left_only,0.000110
3,aiken,South Carolina,South Carolina,2003,10.0,146903.0,NaN,NaN,NaN,NaN,left_only,0.000068
4,alachua,Florida,Florida,2003,11.0,221717.0,NaN,NaN,NaN,NaN,left_only,0.000050
...,...,...,...,...,...,...,...,...,...,...,...,...
40477,sweetwater,Wyoming,NaN,2015,NaN,45604.0,sweetwater,WY,2.412509e+07,17488.438403,both,NaN
40478,teton,Wyoming,NaN,2015,NaN,21800.0,teton,WY,3.343610e+06,2609.264725,both,NaN
40479,uinta,Wyoming,NaN,2015,NaN,21154.0,uinta,WY,1.135771e+07,8459.162985,both,NaN
40480,washakie,Wyoming,NaN,2015,NaN,8586.0,washakie,WY,7.700956e+06,5477.434225,both,NaN


In [55]:
#impute the missing values for death rate based on the avg of that county
#merged['Death Rate'] = merged.groupby(['COUNTY', 'Year'])['Death Rate'].transform(lambda x: x.fillna(x.mean()))

In [56]:
#find alabama in 2003 and sort by death rate
merged[(merged['STATE'] == 'Alabama') & (merged['Year'] == 2003)].sort_values(by='Death Rate', ascending=False)

#find the avg death rate for states in a particular year and assign that to the missing values
merged['Death Rate'] = merged.groupby(['STATE', 'Year'])['Death Rate'].transform(lambda x: x.fillna(x.mean()))

In [57]:
#count unique buyer-county-year rows in shipments
##shipments.groupby(['BUYER_COUNTY', 'BUYER_STATE', 'Year']).size().reset_index(name='count').sort_values(by='count', ascending=False)

In [58]:
#check how many MME columns are null and after 2006 and count unique buyer-county-year rows in shipments
merged

#which counties have null MME


,COUNTY,STATE,State Name_x,Year,Deaths,Population,BUYER_COUNTY,BUYER_STATE,MME,CALC_BASE_WT_IN_GM,merge_indicator,Death Rate
0,acadia,Louisiana,Louisiana,2003,11.0,59209.0,NaN,NaN,NaN,NaN,left_only,0.000186
1,ada,Idaho,Idaho,2003,17.0,325482.0,NaN,NaN,NaN,NaN,left_only,0.000052
2,adams,Colorado,Colorado,2003,42.0,381370.0,NaN,NaN,NaN,NaN,left_only,0.000110
3,aiken,South Carolina,South Carolina,2003,10.0,146903.0,NaN,NaN,NaN,NaN,left_only,0.000068
4,alachua,Florida,Florida,2003,11.0,221717.0,NaN,NaN,NaN,NaN,left_only,0.000050
...,...,...,...,...,...,...,...,...,...,...,...,...
40477,sweetwater,Wyoming,NaN,2015,NaN,45604.0,sweetwater,WY,2.412509e+07,17488.438403,both,0.000178
40478,teton,Wyoming,NaN,2015,NaN,21800.0,teton,WY,3.343610e+06,2609.264725,both,0.000178
40479,uinta,Wyoming,NaN,2015,NaN,21154.0,uinta,WY,1.135771e+07,8459.162985,both,0.000178
40480,washakie,Wyoming,NaN,2015,NaN,8586.0,washakie,WY,7.700956e+06,5477.434225,both,0.000178


In [59]:
#where MME is null

In [60]:
#shipments


In [61]:
#shipments.isna().sum()


In [62]:
#calculate MME rate
merged['MME Rate'] = merged['MME']/merged['Population']

In [63]:
merged

,COUNTY,STATE,State Name_x,Year,Deaths,Population,BUYER_COUNTY,BUYER_STATE,MME,CALC_BASE_WT_IN_GM,merge_indicator,Death Rate,MME Rate
0,acadia,Louisiana,Louisiana,2003,11.0,59209.0,NaN,NaN,NaN,NaN,left_only,0.000186,NaN
1,ada,Idaho,Idaho,2003,17.0,325482.0,NaN,NaN,NaN,NaN,left_only,0.000052,NaN
2,adams,Colorado,Colorado,2003,42.0,381370.0,NaN,NaN,NaN,NaN,left_only,0.000110,NaN
3,aiken,South Carolina,South Carolina,2003,10.0,146903.0,NaN,NaN,NaN,NaN,left_only,0.000068,NaN
4,alachua,Florida,Florida,2003,11.0,221717.0,NaN,NaN,NaN,NaN,left_only,0.000050,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40477,sweetwater,Wyoming,NaN,2015,NaN,45604.0,sweetwater,WY,2.412509e+07,17488.438403,both,0.000178,529.012668
40478,teton,Wyoming,NaN,2015,NaN,21800.0,teton,WY,3.343610e+06,2609.264725,both,0.000178,153.376619
40479,uinta,Wyoming,NaN,2015,NaN,21154.0,uinta,WY,1.135771e+07,8459.162985,both,0.000178,536.906119
40480,washakie,Wyoming,NaN,2015,NaN,8586.0,washakie,WY,7.700956e+06,5477.434225,both,0.000178,896.920095


In [64]:
merged['MME Rate'] = merged.groupby(['STATE', 'Year'])['MME Rate'].transform(lambda x: x.fillna(x.mean()))

In [67]:
#save merged file as csv
merged.to_csv('../20_Merge_Data/merged.csv', index=False)

In [66]:
merged

,COUNTY,STATE,State Name_x,Year,Deaths,Population,BUYER_COUNTY,BUYER_STATE,MME,CALC_BASE_WT_IN_GM,merge_indicator,Death Rate,MME Rate
0,acadia,Louisiana,Louisiana,2003,11.0,59209.0,NaN,NaN,NaN,NaN,left_only,0.000186,NaN
1,ada,Idaho,Idaho,2003,17.0,325482.0,NaN,NaN,NaN,NaN,left_only,0.000052,NaN
2,adams,Colorado,Colorado,2003,42.0,381370.0,NaN,NaN,NaN,NaN,left_only,0.000110,NaN
3,aiken,South Carolina,South Carolina,2003,10.0,146903.0,NaN,NaN,NaN,NaN,left_only,0.000068,NaN
4,alachua,Florida,Florida,2003,11.0,221717.0,NaN,NaN,NaN,NaN,left_only,0.000050,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40477,sweetwater,Wyoming,NaN,2015,NaN,45604.0,sweetwater,WY,2.412509e+07,17488.438403,both,0.000178,529.012668
40478,teton,Wyoming,NaN,2015,NaN,21800.0,teton,WY,3.343610e+06,2609.264725,both,0.000178,153.376619
40479,uinta,Wyoming,NaN,2015,NaN,21154.0,uinta,WY,1.135771e+07,8459.162985,both,0.000178,536.906119
40480,washakie,Wyoming,NaN,2015,NaN,8586.0,washakie,WY,7.700956e+06,5477.434225,both,0.000178,896.920095
